# سادہ امیج کلاسفائر

یہ نوٹ بک آپ کو دکھاتی ہے کہ پہلے سے تربیت یافتہ نیورل نیٹ ورک کا استعمال کرتے ہوئے تصاویر کو کیسے درجہ بند کیا جائے۔

**آپ کیا سیکھیں گے:**
- پہلے سے تربیت یافتہ ماڈل کو لوڈ اور استعمال کرنے کا طریقہ
- تصویر کی پیشگی پروسیسنگ
- تصاویر پر پیش گوئیاں کرنا
- اعتماد کے اسکورز کو سمجھنا

**استعمال کا کیس:** تصاویر میں اشیاء کی شناخت کریں (جیسے "بلی"، "کتا"، "گاڑی" وغیرہ) 

---


## مرحلہ 1: ضروری لائبریریاں درآمد کریں

آئیے وہ اوزار درآمد کریں جن کی ہمیں ضرورت ہے۔ فکر نہ کریں اگر آپ ابھی ان سب کو نہیں سمجھتے!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## مرحلہ 2: پہلے سے تربیت یافتہ ماڈل لوڈ کریں

ہم **MobileNetV2** استعمال کریں گے، جو لاکھوں تصاویر پر پہلے ہی تربیت یافتہ نیورل نیٹ ورک ہے۔

اسے **Transfer Learning** کہا جاتا ہے - کسی اور کے تربیت یافتہ ماڈل کا استعمال!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## مرحلہ 3: مددگار فنکشنز

آئیے ایسے فنکشنز بناتے ہیں جو ہمارے ماڈل کے لیے تصاویر کو لوڈ اور تیار کریں۔


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## مرحلہ 4: نمونہ تصاویر پر آزمائش کریں

چلیں انٹرنیٹ سے کچھ تصاویر کی درجہ بندی کرنے کی کوشش کرتے ہیں!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ہر تصویر کو درجہ بندی کریں


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## مرحلہ 5: اپنی تصاویر آزمائیں!

نیچے دی گئی URL کو کسی بھی تصویر کی URL سے تبدیل کریں جسے آپ درجہ بندی کرنا چاہتے ہیں۔


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 کیا ہوا؟

1. **ہم نے ایک پہلے سے تربیت یافتہ ماڈل لوڈ کیا** - MobileNetV2 کو لاکھوں تصاویر پر تربیت دی گئی تھی  
2. **ہم نے تصاویر کو پہلے سے پروسیس کیا** - انہیں ماڈل کے لیے سائز اور فارمیٹ کیا  
3. **ماڈل نے پیش گوئیاں کیں** - اس نے 1000 اشیاء کی کلاسز کے لیے امکانات دیے  
4. **ہم نے نتائج کو ڈی کوڈ کیا** - نمبروں کو انسان کے لیے قابل فہم لیبلز میں تبدیل کیا  

### اعتماد کے اسکور کو سمجھنا

- **90-100%**: بہت زیادہ اعتماد (تقریباً یقینی طور پر درست)  
- **70-90%**: اعتماد (شاید درست)  
- **50-70%**: کچھ حد تک اعتماد (ممکنہ طور پر درست)  
- **50% سے کم**: زیادہ اعتماد نہیں (غیر یقینی)  

### پیش گوئیاں غلط کیوں ہو سکتی ہیں؟

- **غیر معمولی زاویہ یا روشنی** - ماڈل کو عام تصاویر پر تربیت دی گئی تھی  
- **متعدد اشیاء** - ماڈل ایک مرکزی شے کی توقع کرتا ہے  
- **نایاب اشیاء** - ماڈل صرف 1000 زمروں کو جانتا ہے  
- **کم معیار کی تصویر** - دھندلی یا پکسل شدہ تصاویر زیادہ مشکل ہوتی ہیں  

---  


## 🚀 اگلے مراحل

1. **مختلف تصاویر آزمائیں:**
   - [Unsplash](https://unsplash.com) پر تصاویر تلاش کریں
   - دائیں کلک کریں → "Copy image address" سے URL حاصل کریں

2. **تجربہ کریں:**
   - تجریدی آرٹ کے ساتھ کیا ہوتا ہے؟
   - کیا یہ مختلف زاویوں سے اشیاء کو پہچان سکتا ہے؟
   - یہ متعدد اشیاء کو کیسے ہینڈل کرتا ہے؟

3. **مزید سیکھیں:**
   - [Computer Vision کے اسباق](../lessons/4-ComputerVision/README.md) کو دریافت کریں
   - اپنی تصویر کی درجہ بندی کرنے والا ماڈل تربیت دینا سیکھیں
   - سمجھیں کہ CNNs (Convolutional Neural Networks) کیسے کام کرتے ہیں

---

## 🎉 مبارک ہو!

آپ نے ابھی ایک جدید نیورل نیٹ ورک استعمال کرتے ہوئے تصویر کی درجہ بندی کرنے والا ماڈل بنایا ہے!

یہی تکنیک درج ذیل میں استعمال ہوتی ہے:
- گوگل فوٹوز (آپ کی تصاویر کو منظم کرنا)
- خودکار گاڑیاں (اشیاء کو پہچاننا)
- طبی تشخیص (ایکس رے کا تجزیہ کرنا)
- معیار کی جانچ (نقائص کا پتہ لگانا)

سیکھتے رہیں اور دریافت کرتے رہیں! 🚀



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
